# Build a logistic regression between vaccine rate ~ accessibility + IMD decile + ethnic composition

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import mapclassify as mc
from legendgram import legendgram
import palettable.matplotlib as palmpl
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm

## Read data

In [41]:
#Read the fca result and vaccination rate data
fca = pd.read_csv('fca_result_include_London.csv')

In [3]:
# Read the IMD data
imd = pd.read_csv('imd2019_msoa_level_data.csv')
imd.head(2)

,MSOAC,MSOAHOCLN,LAD19C,LAD19N,REG,IMD19 SCORE,MSOADECILE
0,E02004712,Hiltingbury,E07000086,Eastleigh,South East,2.179958,10
1,E02004757,Fleet South & Pondtail,E07000089,Hart,South East,2.281885,10


In [4]:
# Read the Ethnic group data
ethnic = pd.read_csv('ethnic_group_2011.csv')
ethnic.head(2)

,MSOA,White%,Mixed%,Asian%,Black%,Other%
0,E02002559,0.9742,0.0102,0.0116,0.0027,0.0012
1,E02002560,0.9788,0.0088,0.0088,0.0025,0.0010


In [42]:
# Merge data
fca = pd.merge(left=fca, right=imd[['MSOAC','IMD19 SCORE','MSOADECILE']], how='left', left_on='MSOA11CD', right_on='MSOAC')
fca = pd.merge(left=fca, right=ethnic, how='left', left_on='MSOA11CD', right_on='MSOA')
fca = fca.drop(['MSOAC', 'MSOA','pop60+69','60-69','geometry'], axis=1)
fca.head(2)

,LA_Code_2020,LA_name_2020,MSOA11CD,MSOA11NM,vaccination_rate_60+69,2sfca_10_normalized,2sfca_15_normalized,2sfca_20_normalized,2sfca_25_normalized,2sfca_30_normalized,...,3sfca_20_normalized,3sfca_25_normalized,3sfca_30_normalized,IMD19 SCORE,MSOADECILE,White%,Mixed%,Asian%,Black%,Other%
0,E09000001,City of London,E02000001,City of London 001,0.533659,0.941437,0.970288,0.970929,0.990135,0.983475,...,0.956281,0.968526,0.975433,14.720484,7,0.7863,0.0392,0.1275,0.0262,0.0209
1,E09000002,Barking and Dagenham,E02000002,Barking and Dagenham 001,0.629703,0.176925,0.357423,0.705402,0.889726,0.979142,...,0.392617,0.478684,0.540530,34.767560,2,0.6499,0.0487,0.1210,0.1672,0.0131


## Use Variance Inflation Factors (VIF) to drop columns to avoid multicollinearity between predictors

In [ ]:
# calculating VIF
# This function is partially from: https://stackoverflow.com/a/51329496/4667568
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from statsmodels.tools.tools import add_constant

def drop_column_using_vif_(df, thresh=5):
    '''
    Calculates VIF each feature in a pandas dataframe, and repeatedly drop the columns with the highest VIF
    A constant must be added to variance_inflation_factor or the results will be incorrect

    :param df: the pandas dataframe containing only the predictor features, not the response variable
    :param thresh: the max VIF value before the feature is removed from the dataframe
    :return: dataframe with multicollinear features removed
    '''
    while True:
        # adding a constatnt item to the data
        df_with_const = add_constant(df)

        vif_df = pd.Series([variance_inflation_factor(df_with_const.values, i) 
               for i in range(df_with_const.shape[1])], name= "VIF",
              index=df_with_const.columns).to_frame()

        # drop the const
        vif_df = vif_df.drop('const')
        
        # if the largest VIF is above the thresh, remove a variable with the largest VIF
        if vif_df.VIF.max() > thresh:
            # If there are multiple variables with the maximum VIF, choose the first one
            index_to_drop = vif_df.index[vif_df.VIF == vif_df.VIF.max()].tolist()[0]
            print('Dropping: {}'.format(index_to_drop))
            df = df.drop(columns = index_to_drop)
        else:
            # No VIF is above threshold. Exit the loop
            break

    return df

In [11]:
drop_column_using_vif_(df[['IMD19 SCORE','White%','Mixed%','Asian%','Black%','Other%']])

Dropping: White%


,IMD19 SCORE,Mixed%,Asian%,Black%,Other%
0,-0.520182,0.980142,0.456387,-0.101273,0.685929
1,0.997725,1.516705,0.402464,2.129545,0.209527
2,0.269490,1.200415,1.285145,2.033035,0.771437
3,0.149124,0.466171,-0.181565,1.145454,-0.315737
4,0.652864,0.833293,0.415737,2.154860,-0.022566
...,...,...,...,...,...
6786,-0.076305,1.725682,0.202533,0.386027,0.295035
6787,-0.185855,1.889474,0.027490,1.409672,0.307250
6788,0.209840,0.149882,0.857907,-0.121840,1.791426
6789,0.514256,0.918014,0.022513,-0.047480,1.870826


That means we should drop %white column

## Rescale independent variables (normalization)

In [43]:
# Rescale the independent variables
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = fca[['2sfca_10_normalized','2sfca_15_normalized','2sfca_20_normalized','2sfca_25_normalized','2sfca_30_normalized',
         'E2sfca_10_normalized','E2sfca_15_normalized','E2sfca_20_normalized','E2sfca_25_normalized','E2sfca_30_normalized',
         '3sfca_10_normalized','3sfca_15_normalized','3sfca_20_normalized','3sfca_25_normalized','3sfca_30_normalized',
         'IMD19 SCORE','Mixed%','White%','Asian%','Black%','Other%']]
scaled_data = scaler.fit_transform(X)
df = pd.DataFrame(scaled_data)
df.columns=['2sfca_10_normalized','2sfca_15_normalized','2sfca_20_normalized','2sfca_25_normalized','2sfca_30_normalized',
         'E2sfca_10_normalized','E2sfca_15_normalized','E2sfca_20_normalized','E2sfca_25_normalized','E2sfca_30_normalized',
         '3sfca_10_normalized','3sfca_15_normalized','3sfca_20_normalized','3sfca_25_normalized','3sfca_30_normalized',
         'IMD19 SCORE','Mixed%','White%','Asian%','Black%','Other%']
df.head(2)

,2sfca_10_normalized,2sfca_15_normalized,2sfca_20_normalized,2sfca_25_normalized,2sfca_30_normalized,E2sfca_10_normalized,E2sfca_15_normalized,E2sfca_20_normalized,E2sfca_25_normalized,E2sfca_30_normalized,...,3sfca_15_normalized,3sfca_20_normalized,3sfca_25_normalized,3sfca_30_normalized,IMD19 SCORE,Mixed%,White%,Asian%,Black%,Other%
0,3.941531,3.273806,2.69508,2.414192,2.233286,5.353449,4.626677,4.158952,3.807566,3.532377,...,4.179715,3.650030,3.337662,3.144570,-0.520182,0.980142,-0.428349,0.456387,-0.101273,0.685929
1,0.188260,0.714561,1.72728,2.079027,2.219318,0.044205,0.186775,0.432619,0.735207,1.018985,...,0.655171,1.020898,1.218058,1.346838,0.997725,1.516705,-1.186178,0.402464,2.129545,0.209527


# See the linear regression results

## 2SFCA

Prob (F-Statistic) : uses this number to tell you the accuracy of the null hypothesis, or whether it is accurate that your variables’ effect is 0.

In [47]:
# 10mile
x=df[['2sfca_10_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.156
Model:                                OLS   Adj. R-squared:                  0.155
Method:                     Least Squares   F-statistic:                     208.9
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          2.71e-245
Time:                            18:31:34   Log-Likelihood:                 7400.4
No. Observations:                    6791   AIC:                        -1.479e+04
Df Residuals:                        6784   BIC:                        -1.474e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    793.655      0.000       0.782       0.786
2sfca_10_normalized    -0.0151      0.002     -9.983      0.000      -0.018      -0.012
IMD19 SCORE            -0.0256      0.001    -23.829      0.000      -0.028      -0.024
Mixed%                  0.0215      0.002     11.615      0.000       0.018       0.025
Asian%                 -0.0041      0.001     -3.461      0.001      -0.006      -0.002
Black%                 -0.0167      0.002     -9.726      0.000      -0.020      -0.013
Other%                 -0.0077      0.002     -5.005      0.000      -0.011      -0.005
==============================================================================
Omnibus:                      161.779   Durbin-Watson:                   0.836
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              213.417
Skew:                          -0.290   Prob(JB):                     4.54e-47
Kurtosis:                       3.646   Cond. No.                         4.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
# 15mile
x=df[['2sfca_15_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.146
Model:                                OLS   Adj. R-squared:                  0.145
Method:                     Least Squares   F-statistic:                     192.9
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          1.24e-227
Time:                            18:03:10   Log-Likelihood:                 7359.6
No. Observations:                    6791   AIC:                        -1.471e+04
Df Residuals:                        6784   BIC:                        -1.466e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    788.897      0.000       0.782       0.786
2sfca_15_normalized    -0.0067      0.002     -4.167      0.000      -0.010      -0.004
IMD19 SCORE            -0.0244      0.001    -22.080      0.000      -0.027      -0.022
Mixed%                  0.0191      0.002     10.215      0.000       0.015       0.023
Asian%                 -0.0029      0.001     -2.404      0.016      -0.005      -0.001
Black%                 -0.0186      0.002    -10.626      0.000      -0.022      -0.015
Other%                 -0.0112      0.002     -7.343      0.000      -0.014      -0.008
==============================================================================
Omnibus:                      177.121   Durbin-Watson:                   0.819
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              236.402
Skew:                          -0.306   Prob(JB):                     4.63e-52
Kurtosis:                       3.679   Cond. No.                         4.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
# 20mile
x=df[['2sfca_20_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.144
Model:                                OLS   Adj. R-squared:                  0.143
Method:                     Least Squares   F-statistic:                     189.7
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          4.01e-224
Time:                            18:11:28   Log-Likelihood:                 7351.5
No. Observations:                    6791   AIC:                        -1.469e+04
Df Residuals:                        6784   BIC:                        -1.464e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    787.954      0.000       0.782       0.786
2sfca_20_normalized    -0.0016      0.002     -1.059      0.290      -0.005       0.001
IMD19 SCORE            -0.0234      0.001    -20.859      0.000      -0.026      -0.021
Mixed%                  0.0176      0.002      9.417      0.000       0.014       0.021
Asian%                 -0.0027      0.001     -2.304      0.021      -0.005      -0.000
Black%                 -0.0202      0.002    -11.571      0.000      -0.024      -0.017
Other%                 -0.0128      0.001     -8.594      0.000      -0.016      -0.010
==============================================================================
Omnibus:                      190.169   Durbin-Watson:                   0.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              255.676
Skew:                          -0.320   Prob(JB):                     3.03e-56
Kurtosis:                       3.703   Cond. No.                         4.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
# 25mile
x=df[['2sfca_25_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.144
Model:                                OLS   Adj. R-squared:                  0.143
Method:                     Least Squares   F-statistic:                     189.6
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          5.00e-224
Time:                            18:11:58   Log-Likelihood:                 7351.2
No. Observations:                    6791   AIC:                        -1.469e+04
Df Residuals:                        6784   BIC:                        -1.464e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    787.928      0.000       0.782       0.786
2sfca_25_normalized    -0.0012      0.001     -0.823      0.410      -0.004       0.002
IMD19 SCORE            -0.0233      0.001    -20.686      0.000      -0.026      -0.021
Mixed%                  0.0175      0.002      9.376      0.000       0.014       0.021
Asian%                 -0.0027      0.001     -2.286      0.022      -0.005      -0.000
Black%                 -0.0203      0.002    -11.687      0.000      -0.024      -0.017
Other%                 -0.0130      0.001     -8.797      0.000      -0.016      -0.010
==============================================================================
Omnibus:                      191.723   Durbin-Watson:                   0.812
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              258.041
Skew:                          -0.321   Prob(JB):                     9.27e-57
Kurtosis:                       3.707   Cond. No.                         4.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
# 30mile
x=df[['2sfca_30_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.144
Model:                                OLS   Adj. R-squared:                  0.143
Method:                     Least Squares   F-statistic:                     189.9
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          2.11e-224
Time:                            18:12:01   Log-Likelihood:                 7352.1
No. Observations:                    6791   AIC:                        -1.469e+04
Df Residuals:                        6784   BIC:                        -1.464e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    788.029      0.000       0.782       0.786
2sfca_30_normalized    -0.0022      0.001     -1.554      0.120      -0.005       0.001
IMD19 SCORE            -0.0236      0.001    -20.861      0.000      -0.026      -0.021
Mixed%                  0.0178      0.002      9.560      0.000       0.014       0.021
Asian%                 -0.0026      0.001     -2.205      0.027      -0.005      -0.000
Black%                 -0.0200      0.002    -11.586      0.000      -0.023      -0.017
Other%                 -0.0128      0.001     -8.750      0.000      -0.016      -0.010
==============================================================================
Omnibus:                      191.561   Durbin-Watson:                   0.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              258.304
Skew:                          -0.320   Prob(JB):                     8.13e-57
Kurtosis:                       3.709   Cond. No.                         4.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## E2SFCA

In [28]:
# 10mile
x=df[['E2sfca_10_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.167
Model:                                OLS   Adj. R-squared:                  0.166
Method:                     Least Squares   F-statistic:                     226.5
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          1.66e-264
Time:                            18:15:07   Log-Likelihood:                 7444.8
No. Observations:                    6791   AIC:                        -1.488e+04
Df Residuals:                        6784   BIC:                        -1.483e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7841      0.001    798.864      0.000       0.782       0.786
E2sfca_10_normalized    -0.0189      0.001    -13.795      0.000      -0.022      -0.016
IMD19 SCORE             -0.0255      0.001    -24.241      0.000      -0.028      -0.023
Mixed%                   0.0227      0.002     12.401      0.000       0.019       0.026
Asian%                  -0.0048      0.001     -4.069      0.000      -0.007      -0.002
Black%                  -0.0176      0.002    -10.523      0.000      -0.021      -0.014
Other%                  -0.0057      0.002     -3.726      0.000      -0.009      -0.003
==============================================================================
Omnibus:                      158.219   Durbin-Watson:                   0.846
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              211.072
Skew:                          -0.282   Prob(JB):                     1.47e-46
Kurtosis:                       3.654   Cond. No.                         4.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
# 15mile
x=df[['E2sfca_15_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.161
Model:                                OLS   Adj. R-squared:                  0.160
Method:                     Least Squares   F-statistic:                     217.2
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          1.95e-254
Time:                            18:15:11   Log-Likelihood:                 7421.6
No. Observations:                    6791   AIC:                        -1.483e+04
Df Residuals:                        6784   BIC:                        -1.478e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7841      0.001    796.130      0.000       0.782       0.786
E2sfca_15_normalized    -0.0176      0.001    -11.944      0.000      -0.020      -0.015
IMD19 SCORE             -0.0258      0.001    -24.204      0.000      -0.028      -0.024
Mixed%                   0.0225      0.002     12.168      0.000       0.019       0.026
Asian%                  -0.0044      0.001     -3.715      0.000      -0.007      -0.002
Black%                  -0.0168      0.002     -9.910      0.000      -0.020      -0.013
Other%                  -0.0064      0.002     -4.214      0.000      -0.009      -0.003
==============================================================================
Omnibus:                      159.613   Durbin-Watson:                   0.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              211.886
Skew:                          -0.285   Prob(JB):                     9.76e-47
Kurtosis:                       3.650   Cond. No.                         4.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# 20mile
x=df[['E2sfca_20_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.155
Model:                                OLS   Adj. R-squared:                  0.154
Method:                     Least Squares   F-statistic:                     207.5
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          7.74e-244
Time:                            18:15:14   Log-Likelihood:                 7397.0
No. Observations:                    6791   AIC:                        -1.478e+04
Df Residuals:                        6784   BIC:                        -1.473e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7841      0.001    793.262      0.000       0.782       0.786
E2sfca_20_normalized    -0.0149      0.002     -9.635      0.000      -0.018      -0.012
IMD19 SCORE             -0.0257      0.001    -23.786      0.000      -0.028      -0.024
Mixed%                   0.0217      0.002     11.680      0.000       0.018       0.025
Asian%                  -0.0038      0.001     -3.177      0.001      -0.006      -0.001
Black%                  -0.0168      0.002     -9.814      0.000      -0.020      -0.013
Other%                  -0.0078      0.002     -5.074      0.000      -0.011      -0.005
==============================================================================
Omnibus:                      161.657   Durbin-Watson:                   0.835
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              214.350
Skew:                          -0.288   Prob(JB):                     2.85e-47
Kurtosis:                       3.652   Cond. No.                         4.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
# 25mile
x=df[['E2sfca_25_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.151
Model:                                OLS   Adj. R-squared:                  0.150
Method:                     Least Squares   F-statistic:                     200.6
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          3.15e-236
Time:                            18:15:18   Log-Likelihood:                 7379.5
No. Observations:                    6791   AIC:                        -1.474e+04
Df Residuals:                        6784   BIC:                        -1.470e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7841      0.001    791.209      0.000       0.782       0.786
E2sfca_25_normalized    -0.0121      0.002     -7.569      0.000      -0.015      -0.009
IMD19 SCORE             -0.0254      0.001    -23.226      0.000      -0.028      -0.023
Mixed%                   0.0209      0.002     11.172      0.000       0.017       0.025
Asian%                  -0.0033      0.001     -2.759      0.006      -0.006      -0.001
Black%                  -0.0173      0.002    -10.013      0.000      -0.021      -0.014
Other%                  -0.0091      0.002     -5.931      0.000      -0.012      -0.006
==============================================================================
Omnibus:                      166.089   Durbin-Watson:                   0.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              220.629
Skew:                          -0.294   Prob(JB):                     1.23e-48
Kurtosis:                       3.660   Cond. No.                         4.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
# 30mile
x=df[['E2sfca_30_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.148
Model:                                OLS   Adj. R-squared:                  0.147
Method:                     Least Squares   F-statistic:                     196.4
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          1.34e-231
Time:                            18:15:21   Log-Likelihood:                 7368.7
No. Observations:                    6791   AIC:                        -1.472e+04
Df Residuals:                        6784   BIC:                        -1.468e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.7841      0.001    789.962      0.000       0.782       0.786
E2sfca_30_normalized    -0.0096      0.002     -5.980      0.000      -0.013      -0.006
IMD19 SCORE             -0.0251      0.001    -22.697      0.000      -0.027      -0.023
Mixed%                   0.0202      0.002     10.751      0.000       0.016       0.024
Asian%                  -0.0030      0.001     -2.501      0.012      -0.005      -0.001
Black%                  -0.0179      0.002    -10.287      0.000      -0.021      -0.014
Other%                  -0.0101      0.002     -6.631      0.000      -0.013      -0.007
==============================================================================
Omnibus:                      171.380   Durbin-Watson:                   0.824
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              228.349
Skew:                          -0.299   Prob(JB):                     2.60e-50
Kurtosis:                       3.670   Cond. No.                         4.21
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 3SFCA

In [33]:
# 10mile
x=df[['3sfca_10_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.161
Model:                                OLS   Adj. R-squared:                  0.160
Method:                     Least Squares   F-statistic:                     216.4
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          1.40e-253
Time:                            18:17:31   Log-Likelihood:                 7419.6
No. Observations:                    6791   AIC:                        -1.483e+04
Df Residuals:                        6784   BIC:                        -1.478e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    795.898      0.000       0.782       0.786
3sfca_10_normalized    -0.0167      0.001    -11.774      0.000      -0.019      -0.014
IMD19 SCORE            -0.0257      0.001    -24.119      0.000      -0.028      -0.024
Mixed%                  0.0221      0.002     12.002      0.000       0.018       0.026
Asian%                 -0.0042      0.001     -3.578      0.000      -0.007      -0.002
Black%                 -0.0175      0.002    -10.394      0.000      -0.021      -0.014
Other%                 -0.0067      0.002     -4.418      0.000      -0.010      -0.004
==============================================================================
Omnibus:                      157.440   Durbin-Watson:                   0.840
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              209.215
Skew:                          -0.282   Prob(JB):                     3.71e-46
Kurtosis:                       3.649   Cond. No.                         4.16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
# 15mile
x=df[['3sfca_15_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.155
Model:                                OLS   Adj. R-squared:                  0.154
Method:                     Least Squares   F-statistic:                     206.7
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          6.29e-243
Time:                            18:17:31   Log-Likelihood:                 7394.9
No. Observations:                    6791   AIC:                        -1.478e+04
Df Residuals:                        6784   BIC:                        -1.473e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    793.016      0.000       0.782       0.786
3sfca_15_normalized    -0.0143      0.002     -9.412      0.000      -0.017      -0.011
IMD19 SCORE            -0.0258      0.001    -23.776      0.000      -0.028      -0.024
Mixed%                  0.0216      0.002     11.608      0.000       0.018       0.025
Asian%                 -0.0036      0.001     -3.043      0.002      -0.006      -0.001
Black%                 -0.0173      0.002    -10.119      0.000      -0.021      -0.014
Other%                 -0.0081      0.002     -5.268      0.000      -0.011      -0.005
==============================================================================
Omnibus:                      163.230   Durbin-Watson:                   0.833
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              217.421
Skew:                          -0.289   Prob(JB):                     6.13e-48
Kurtosis:                       3.659   Cond. No.                         4.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
# 20mile
x=df[['3sfca_20_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.151
Model:                                OLS   Adj. R-squared:                  0.150
Method:                     Least Squares   F-statistic:                     200.6
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          2.95e-236
Time:                            18:17:31   Log-Likelihood:                 7379.5
No. Observations:                    6791   AIC:                        -1.475e+04
Df Residuals:                        6784   BIC:                        -1.470e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    791.217      0.000       0.782       0.786
3sfca_20_normalized    -0.0119      0.002     -7.578      0.000      -0.015      -0.009
IMD19 SCORE            -0.0257      0.001    -23.241      0.000      -0.028      -0.023
Mixed%                  0.0209      0.002     11.170      0.000       0.017       0.025
Asian%                 -0.0031      0.001     -2.658      0.008      -0.005      -0.001
Black%                 -0.0174      0.002    -10.116      0.000      -0.021      -0.014
Other%                 -0.0093      0.002     -6.113      0.000      -0.012      -0.006
==============================================================================
Omnibus:                      169.263   Durbin-Watson:                   0.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              226.305
Skew:                          -0.296   Prob(JB):                     7.22e-50
Kurtosis:                       3.671   Cond. No.                         4.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
# 25mile
x=df[['3sfca_25_normalized','IMD19 SCORE','Mixed%','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.149
Model:                                OLS   Adj. R-squared:                  0.148
Method:                     Least Squares   F-statistic:                     197.6
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          7.12e-233
Time:                            18:17:31   Log-Likelihood:                 7371.7
No. Observations:                    6791   AIC:                        -1.473e+04
Df Residuals:                        6784   BIC:                        -1.468e+04
Df Model:                               6                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    790.306      0.000       0.782       0.786
3sfca_25_normalized    -0.0102      0.002     -6.456      0.000      -0.013      -0.007
IMD19 SCORE            -0.0255      0.001    -22.822      0.000      -0.028      -0.023
Mixed%                  0.0204      0.002     10.874      0.000       0.017       0.024
Asian%                 -0.0029      0.001     -2.439      0.015      -0.005      -0.001
Black%                 -0.0177      0.002    -10.215      0.000      -0.021      -0.014
Other%                 -0.0101      0.002     -6.678      0.000      -0.013      -0.007
==============================================================================
Omnibus:                      173.607   Durbin-Watson:                   0.824
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              232.844
Skew:                          -0.300   Prob(JB):                     2.74e-51
Kurtosis:                       3.680   Cond. No.                         4.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# 30mile
x=df[['3sfca_30_normalized','IMD19 SCORE','Asian%','Black%','Other%']]
y=fca['vaccination_rate_60+69']
regressor_OLS = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                              
==================================================================================
Dep. Variable:     vaccination_rate_60+69   R-squared:                       0.133
Model:                                OLS   Adj. R-squared:                  0.133
Method:                     Least Squares   F-statistic:                     208.9
Date:                    Sat, 26 Jun 2021   Prob (F-statistic):          7.91e-208
Time:                            18:28:22   Log-Likelihood:                 7311.0
No. Observations:                    6791   AIC:                        -1.461e+04
Df Residuals:                        6785   BIC:                        -1.457e+04
Df Model:                               5                                         
Covariance Type:                nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   0.7841      0.001    783.334      0.000       0.782       0.786
3sfca_30_normalized    -0.0047      0.002     -3.046      0.002      -0.008      -0.002
IMD19 SCORE            -0.0234      0.001    -20.943      0.000      -0.026      -0.021
Asian%                 -0.0017      0.001     -1.398      0.162      -0.004       0.001
Black%                 -0.0080      0.001     -5.414      0.000      -0.011      -0.005
Other%                 -0.0069      0.001     -4.658      0.000      -0.010      -0.004
==============================================================================
Omnibus:                      175.880   Durbin-Watson:                   0.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              233.695
Skew:                          -0.306   Prob(JB):                     1.79e-51
Kurtosis:                       3.672   Cond. No.                         3.19
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""